# Corrección de nombres de las estructuras
--------

In [1]:
import pydicom as dcm
import xml.etree.ElementTree as ET
import pandas as pd
from textdistance import ratcliff_obershelp

Las estructuras son creadas por el programa de segmentación del CT, por los Técnicos de simulación o por las Oncólogas de Radioterapia.

Existen plantillas de contorneo pero técnicos y oncólogas son libres de introducir nuuevos nombres o cambiar los existentes.

Extraer los nombres de las estructuras del archivo DICOM exportado de ARIA. Almacenar el resultado en la lista `contstrnames`

In [2]:
dcmds = dcm.read_file('RS.1.2.246.352.221.550136246713487469715724200262812558270.dcm')
strsetsq = dcmds.StructureSetROISequence
contstrnames = [structure.ROIName for structure in strsetsq]

Comprobar que la longitud del nombre de las estructuras es menor de 16 caracteres. Esta una condición requerida por ARIA que utiliza el nombre como identificador de la estructura en el protocolo clínico.

In [3]:
for strname in contstrnames:
    if len(strname) > 16:
        print(strname)

Extraer los nombres de las estructuras según aparecen en la prescripción.

Hacerlo mediante el procesado del protocolo clínico creado a partir de la prescripción

In [4]:
tree = ET.parse('../protocolos/clinicos/ClinicalProtocol.xml')
root = tree.getroot()
structures = root.find('StructureTemplate').find('Structures')
protstrnames = [structure.get('ID') for structure in structures.findall('Structure')]

Hacer el producto cartesiano de distancias entre ambos conjuntos de estructuras. Almacenar el resultado en un DataFrame

In [5]:
infstrdf = pd.DataFrame([{strname: ratcliff_obershelp(protstrname, strname) 
                          for strname in contstrnames} 
                        for protstrname in protstrnames], index=protstrnames)

Añadir una columna con el nombre de la estructura con la mayor similitud

In [6]:
infstrdf['Suggestion'] = infstrdf.idxmax(axis=1)
infstrdf['Suggestion']

PTV LSI                      PTV LSI
Ambos pulmones        Ambos pulmones
Médula                        Médula
via aerea                  via aerea
Tráquea                      Tráquea
Esófago                      Esófago
Corazón                      Corazón
aorta                        Corazón
v cava inferior        CouchInterior
Vena cava sup          Vena cava sup
Arteria Pulmonar    Arteria Pulmonar
venas pulmonares      Ambos pulmones
Piel                            Piel
Pared costal            Pared costal
Hígado                        Hígado
plexo braquial               Tráquea
Name: Suggestion, dtype: object